In [ ]:
# First reset the pygkyl library
import sys
!rm -rf ~/personal_gkyl_scripts/pygkyl/pygkyl.egg-info
!rm -rf ~/personal_gkyl_scripts/pygkyl/build
!{sys.executable} -m pip install ~/personal_gkyl_scripts/pygkyl > ~/personal_gkyl_scripts/pygkyl/install.log

import numpy as np
import matplotlib.pyplot as plt
import os

# Configure plotting
plt.rcParams["figure.figsize"] = (6,4)

# Custom libraries and routines
import pygkyl

home_dir = os.path.expanduser("~")

simdir = '/Users/ahoffman/personal_gkyl_scripts/sim_data_dir_example/3x2v_example/gk_tcv_NT_coarse_3x2v'
fileprefix = 'gk_tcv_negD_trueq_iwl_3x2v'
simulation = pygkyl.Simulation(dimensionality='3x2v')
simulation.set_phys_param(
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
def qprofile(R):
    a = [497.3420166252413, -1408.736172826569, 1331.4134861681464, -419.00692601227627]
    return a[0]*R**3 + a[1]*R**2 + a[2]*R + a[3]
simulation.set_geom_param(
    B_axis      = 1.4,           # Magnetic field at magnetic axis [T]
    R_axis      = 0.8867856264,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0870056099999,   # Major radius of LCFS at the midplane
    a_shift     = 1.0,                 # Parameter in Shafranov shift
    kappa       = 1.4,                 # Elongation factor
    delta       =-0.39,                 # Triangularity factor
    qprofile    = qprofile,                 # Safety factor
    x_LCFS      = 0.04,                 # position of the LCFS (= core domain width)
    x_out       = 0.08                  # SOL domain width
)

# Define the species
ion = pygkyl.Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = pygkyl.Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
# Add them to the simulation (we need to know this before setting up the data parameters)
simulation.add_species(ion)
simulation.add_species(elc)
# This call will set up the data structure of the simulation and set up a large dictionary 
# conaining the receipes of many post processing quantities, see simulation.data_param.info()
simulation.set_data_param( simdir = simdir, fileprefix = fileprefix, species = simulation.species)
# This is the first call that will load data. 
# If the paths are not set up correctly, the script will fail here.
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

simulation.normalization.set('t','mus') # time in micro-seconds
# simulation.normalization.set('x','minor radius') # radial coordinate normalized by the minor radius (rho=r/a)
simulation.normalization.set('y','Larmor radius') # binormal in term of reference sound Larmor radius
simulation.normalization.set('z','pi') # parallel angle devided by pi
simulation.normalization.set('fluid velocities','thermal velocity') # fluid velocity moments are normalized by the thermal velocity
simulation.normalization.set('temperatures','eV') # temperatures in electron Volt
simulation.normalization.set('pressures','Pa') # pressures in Pascal
simulation.normalization.set('energies','MJ') # energies in mega Joules

fieldname = fileprefix+'-field' #e.g. we check the electrostatic field files.
sim_frames = pygkyl.file_utils.find_available_frames(simulation,fieldname)
print("Time frames available from %g to %g"%(sim_frames[0],sim_frames[-1]))

# pygkyl.plot_utils.plot_integrated_moment(simulation,fieldnames=['ntot','Wtot']);

In [ ]:
time = []
P_iw = []
P_ow = []
P_TSlo = []
P_TSup = []
P_limlo = []
P_limup = []
P_ylo   = []
P_yup   = []

# for tf in sim_frames[-1]:
tf = sim_frames[-1]

# radial inner wall loss
f_ = pygkyl.Frame(simulation,'ExB_hflux_x',tf=tf,load=True)
flux_ = f_.compute_surface_integral(
    direction='yz',
    ccoord=0,
    int_bounds=['all','all'],
    integrant_filter='all'
    )
P_iw.append(flux_)

# radial outer wall loss
f_ = pygkyl.Frame(simulation,'ExB_hflux_x',tf=tf,load=True)
flux_ = f_.compute_surface_integral(
    direction='yz',
    ccoord=-1,
    int_bounds=['all','all'],
    integrant_filter='all'
    )
P_ow.append(flux_)

# limiter lower loss
f_ = pygkyl.Frame(simulation,'ExB_hflux_z',tf=tf,load=True)
flux_ = f_.compute_surface_integral(
    direction='xy', 
    ccoord=0,
    int_bounds=[[simulation.geom_param.x_LCFS,0.12],'all'],
    integrant_filter='all'
    )
P_limlo.append(flux_)

# limiter upper loss
f_ = pygkyl.Frame(simulation,'ExB_hflux_z',tf=tf,load=True)
flux_ = f_.compute_surface_integral(
    direction='xy',
    ccoord=-1,
    int_bounds=[[simulation.geom_param.x_LCFS,0.12],'all'],
    integrant_filter='all'
    )
P_limup.append(flux_)

# save time
time.append(f_.time)

# Display the energy bilan computed above
print('Time: %g s'%(time[-1]))
print('Power loss at the inner wall: %g MW'%(P_iw[-1]/1e6))
print('Power loss at the outer wall: %g MW'%(P_ow[-1]/1e6))
print('Power loss at the lower limiter: %g MW'%(P_limlo[-1]/1e6))
print('Power loss at the upper limiter: %g MW'%(P_limup[-1]/1e6))
print('Total power loss: %g MW'%(P_iw[-1]/1e6+P_ow[-1]/1e6+P_limlo[-1]/1e6+P_limup[-1]/1e6))

In [ ]:
# power through the last closed flux surface
time = []
P_LCFS = []

# for tf in sim_frames[-1]:
tf = sim_frames[-1]

# radial inner wall loss
f_ = pygkyl.Frame(simulation,'ExB_hflux_x',tf=tf,load=True)
flux_ = f_.compute_surface_integral(
    direction='yz',
    ccoord=0.04,
    int_bounds=['all','all'],
    integrant_filter='all'
    )
P_iw.append(flux_)

print('Power through the LCFS: %g MW'%(P_iw[-1]/1e6))

In [ ]:
pygkyl.plot_utils.plot_1D(simulation=simulation, fieldnames=[['ExB_hflux_xe','ExB_hflux_xi']], cdirection='x', time_frames=sim_frames[-50:], ccoords=['int','int'])

In [ ]:
def symlog(x):
    """ Returns the symmetric log10 value """
    return np.sign(x) * np.log10(np.abs(x))
plane = 'xy'
ccord = 'int'
fx = pygkyl.Frame(simulation,'ExB_hflux_x',tf=tf,load=True)
fx.slice(plane,ccoord=ccord)
fy = pygkyl.Frame(simulation,'ExB_hflux_y',tf=tf,load=True)
fy.slice(plane,ccoord=ccord)

print(np.shape(fx.values))

[XX,YY] = pygkyl.math_tools.custom_meshgrid(fx.grids[0][:-1],fx.grids[1][:-1])
print(np.shape(XX))
FX = np.squeeze(fx.values)
FY = np.squeeze(fy.values)

fig, ax = plt.subplots()
q = ax.quiver(XX,YY,FX,FY)
# ax.set_xlim([0.04,0.12])
plt.tight_layout()
plt.show()

In [ ]:
phi = pygkyl.Frame(simulation,'phi',tf=tf,load=True)
ni = pygkyl.Frame(simulation,'ni',tf=tf,load=True)
ne = pygkyl.Frame(simulation,'ne',tf=tf,load=True)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(ni.values,phi.values,marker='.')
plt.show()